### Import Packages

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import preprocessing, utils, losses, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import yaml
import numpy as np
from pickle import dump, load
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Load Data & Process

In [2]:
def load_conversation_data(data_directory):
    fileList = os.listdir(data_directory + "/")
    questions = []
    answers = []
    for file in fileList:
        data = yaml.safe_load(open(data_directory + "/" + file, "rb"))
        conversation_list = data["conversations"]
        for conversation in conversation_list:
            for i in range(len(conversation) - 1):
                questions.append(conversation[i])
                answers.append(conversation[i + 1])
    answers_index = []
    for i in range(len(answers)):
        answers_index.append(i)
    return questions, answers, np.array(answers_index)

# load data from all files
questions, answers, answers_index = load_conversation_data("./English")

In [3]:
def vectorize(data):
    if type(data) == str:
       data = [data]
    return tokenizer.texts_to_sequences(data)

def devectorize(data):
    return answers[np.argmax(data)]

# define and fit tokenizer on questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions)

x = pad_sequences(vectorize(questions))
y = answers_index
sentence_maxlen = len(x[0])
vocabulary_size = len(tokenizer.word_index) + 1

### Machine Learning Algorithms

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

models = [
    DecisionTreeClassifier(random_state=0),
    LogisticRegression(random_state=0)
]

# Precision, Recall and F1 Score
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split

#For Each Algorithm 
accuracyList = []
for model in models:
    modelName = model.__class__.__name__
    #Split Data 
    X_train, X_test, y_train, y_test = train_test_split(x, answers, test_size=0.33, random_state=0)
    #Train Algorithm
    model.fit(X_train, y_train)
    # Make Predictions
    y_pred_proba = model.predict_proba(X_test)
    y_pred = model.predict(X_test)
    accuracyList.append([modelName, precision_score(y_test, y_pred, average='macro'), recall_score(y_test, y_pred, average='macro'), f1_score(y_test, y_pred, average='macro')])

pd.DataFrame(accuracyList, columns=["Algorithm", "Precision", "Recall", "F1 Score"])

/home/sheikh/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sheikh/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sheikh/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver opt

,Algorithm,Precision,Recall,F1 Score
0,DecisionTreeClassifier,0.001678,0.004474,0.002386
1,LogisticRegression,0.000840,0.002519,0.001259


### Deep Learning Algorithms

In [50]:
model = Sequential()
model.add(Embedding(input_dim  = vocabulary_size, output_dim = y.shape[0], input_length = sentence_maxlen))
model.add(Dropout(0.3))
model.add(LSTM(len(y), input_shape=x.shape, return_sequences=True))
model.add(Dropout(0.3))
model.add(Dense(len(y), activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 30, 869)           850751    
_________________________________________________________________
dropout_40 (Dropout)         (None, 30, 869)           0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 30, 869)           6044764   
_________________________________________________________________
dropout_41 (Dropout)         (None, 30, 869)           0         
_________________________________________________________________
dense_19 (Dense)             (None, 30, 869)           756030    
Total params: 7,651,545
Trainable params: 7,651,545
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
y.shape

(869,)

In [9]:
x.shape

(869, 30)

In [51]:
# fit the model
model.fit(x,
          y,
          batch_size=100,
          epochs=1000,
          verbose=1,
          validation_split=0.1)

Epoch 1/1000


ValueError: in user code:

    /home/sheikh/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/sheikh/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/sheikh/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/sheikh/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/sheikh/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
        loss = self.compiled_loss(
    /home/sheikh/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/compile_utils.py:205 __call__
    

In [ ]:
# save the trained model
model.save('model.h5')

# pickle the extra files
with open('extrafiles.pkl', 'wb') as f:
    dump([tokenizer, sentence_maxlen, answers], f)
print('Model and extra files saved successfully!')

### Load the Saved Model and Test

In [ ]:
# load the model
model = keras.models.load_model('model.h5')

# load the extra files
with open('extrafiles.pkl', 'rb') as f:
    tokenizer, sentence_maxlen, answers = load(f)

In [ ]:
def get_response(query):
    pad_que = pad_sequences(vectorize(query), maxlen=30)
    return devectorize(model.predict(pad_que))

while True:
    input_query = input()
    if input_query == '':
        break
    else:
        print(' - ' + get_response(input_query))